<a href="https://colab.research.google.com/github/OshadhaAK/ML_Competition/blob/master/best_gradient_boost_with_StratifiedKFold_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import f1_score
import warnings
warnings.simplefilter('ignore')
import pandas.testing as tm

In [ ]:
df = pd.read_csv('/content/drive/My Drive/ml/train.csv',index_col='tripid')
Xtest = pd.read_csv('/content/drive/My Drive/ml/test.csv',index_col='tripid')
submission_df = pd.read_csv("/content/drive/My Drive/ml/sample_submission.csv",index_col='tripid')

In [ ]:
df

In [ ]:
df['label'] = [0 if x == "incorrect" else 1 for x in df['label']]

In [ ]:
import numpy as np
def getDistanceFromLatLonInKm(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2

    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c

    return distance

In [ ]:
df['distance'] = getDistanceFromLatLonInKm(df['drop_lat'],df['drop_lon'],df['pick_lat'],df['pick_lon'])
Xtest['distance'] = getDistanceFromLatLonInKm(Xtest['drop_lat'],Xtest['drop_lon'],Xtest['pick_lat'],Xtest['pick_lon'])

In [ ]:
df = df.drop(['drop_time','pickup_time','drop_lat','drop_lon','pick_lat','pick_lon'], axis = 1)
Xtest = Xtest.drop(['drop_time','pickup_time','drop_lat','drop_lon','pick_lat','pick_lon'], axis = 1)

In [ ]:
np.testing.assert_array_equal(Xtest.index.values, submission_df.index.values)

In [ ]:
df.dropna(how='any', inplace=True)

In [ ]:
X = df.drop('label',1)
y = df.label

In [ ]:
print (y.value_counts())

1    15442
0     1526
Name: label, dtype: int64


In [ ]:
X

In [ ]:
best_three_folds = []
skfold = StratifiedKFold(n_splits=3, shuffle=True)
model = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.05)
model_results = []

for train_index, test_index in skfold.split(X, y):
  X_train, X_test = X.iloc[train_index,:],X.iloc[test_index,:]
  y_train, y_test = y.iloc[train_index],y.iloc[test_index]
  model.fit(X_train,y_train)
  y_pred = model.predict(X_test)
  f1 = f1_score(y_test,y_pred)
  model_results.append(f1)

  prediction = model.predict(Xtest)
  best_three_folds.append(prediction.tolist())

print(model_results,np.mean(model_results))


In [ ]:
best_three_folds = np.array(best_three_folds)
best_three_folds = np.mean(best_three_folds, axis=0)


In [ ]:
ynew = np.where(best_three_folds < 0.5, 0, 1)

In [ ]:
ynew

In [ ]:
import collections
print (collections.Counter(ynew))

In [ ]:
res = pd.DataFrame({'tripid': Xtest.index, 'prediction': ynew})
res.to_csv("/content/drive/My Drive/ml/grahourmin.csv", index=False)

In [ ]:
!head '/content/drive/My Drive/ml/grahourmin.csv'